In [182]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

# set env
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)


from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

data_root = '../data/_datasets/cluster_res'

# data_file = 'seq_dg_cluster_final_230607_v3.csv'
# data_file = os.path.join(data_root, data_file)
# main_df = pd.read_csv(data_file, sep='\t', low_memory=False)
# print(len(main_df))
# main_df.head(6)

mainf = 'seq_dg_230620.txt'
mainf = os.path.join(data_root, mainf)
madf = pd.read_csv(mainf, sep='\t', low_memory=False)
print(len(madf))
# madf.head(3)

madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))

# seq_dg_230620.txt 是大文件;
# dataset_new 是更新后，各个核酸复合物的数据集文件;
# cluster_new 是聚类文件,wt 数据集直接将 uniprot+na连接表示一个复合物，
# mut 将 uniprot+protein_mutation+na 表示复合物。
# 单双链的na不同，单链为 na_index列,双链为 newnafea_na_job 列;

# mutation_type
# UniProt
# nucleic_acid_type_new
# double: newnafea_na_job
# single: na_index

wtf = 'wt.tsv'
wtdf = pd.read_csv( os.path.join(data_root, wtf), sep='\t' )
print(len(wtdf))
# wtdf.head(3)
# protein_sequence nucleotide_sequence dG complex_id

wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))



# check valid
# for i in tqdm(range(len(wtdf))):
#     row = wtdf.loc[i]
#     key = row['protein_sequence'] + row['nucleotide_sequence']
#     cid = row['complex_id']
#     tmp = madf[madf['complex_id'] == cid]
#     key2 = tmp['protein_sequence'].iloc[0] + tmp['nucleotide_sequence'].iloc[0]
#     if key != key2:
#         print(cid)
#     if len(tmp) != 1:
#         print(cid)

17199
4846


array([    0,     1,     2, ..., 17258, 17259, 17260])

In [146]:
# 相同蛋白序列但是UniProt ID 不同：
# P0A6X3,A0A140NGK1,C1IFD2
# P24042,Q71TA8
# Q8DQG2,A0A0H2UPA7
# P0AGK8 C3SZN7

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))

# data_dir = '../data/_datasets/'
# data_file = 'seq_dg_cluster_final_230607_v3.csv'
# data_file = os.path.join(data_dir, data_file)
# main_df.to_csv(data_file, index=False, sep='\t')


# madf[madf['mutation_type'] == 'WILD']

In [185]:
def createMap(data_root, fpath, prefix, output):
    df = pd.read_csv( os.path.join(data_root, fpath), sep='\t')
    for i in range(len(df)):
        row = df.loc[i]
        keys = row['cluster_member'].split(',')
        for k in keys:
            output[k] = prefix + row['cluster_index']

cid2wt = dict()
for c in madf.loc[:, 'complex_id'].values:
    cid2wt[c] = False


pwtc_f = 'protein_wt_cluster.tsv'
# pwtc_f = os.path.join(data_root, pwtc_f)
# pwtcdf = pd.read_csv(pwtc_f, sep='\t')
# cluster_index  cluster_number  cluster_member

unp2cls = dict()
# for i in range(len(pwtcdf)):
#     row = pwtcdf.loc[i]
#     unps = row['cluster_member'].split(',')
#     for unp in unps:
#         unp2cls[unp] = 'P' + row['cluster_index']
createMap(data_root, pwtc_f, 'P', unp2cls)
# madf.insert(len(madf.columns), 'pclass', pclass_list, allow_duplicates=False)

if 'pclass' not in wtdf.columns:
    wtdf.insert(len(wtdf.columns), 'pclass', 'None', allow_duplicates=False)
# # wtdf
for i in range(len(wtdf)):
    row = wtdf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in wt data: ", i, row['complex_id'], row['UniProt'])
    else:
        wtdf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]
    #
    
#


# ddna_df = pd.read_csv( os.path.join(data_root, 'dsDNA_80_cluster.txt'), sep='\t')
# drna_df = pd.read_csv( os.path.join(data_root, 'dsRNA_80_cluster.txt'), sep='\t')
# sdna_df = pd.read_csv( os.path.join(data_root, 'ssDNA_80_cluster.txt'), sep='\t')
# srna_df = pd.read_csv( os.path.join(data_root, 'ssRNA_80_cluster.txt'), sep='\t')

ddna2cls, drna2cls, sdna2cls, srna2cls = dict(), dict(), dict(), dict()
createMap(data_root, 'dsDNA_80_cluster.txt', 'DDNA', ddna2cls)
createMap(data_root, 'dsRNA_80_cluster.txt', 'DRNA', drna2cls)
createMap(data_root, 'ssDNA_80_cluster.txt', 'SDNA', sdna2cls)
createMap(data_root, 'ssRNA_80_cluster.txt', 'SRNA', srna2cls)


if 'pclass' not in madf.columns:
    madf.insert(len(madf.columns), 'pclass', 'None', allow_duplicates=False)
for i in range(len(madf)):
    row = madf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in ma data: ", i, row['complex_id'], row['UniProt'])
    else:
        madf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]

        



In [191]:
cid2wt = dict()
for c in madf.loc[:, 'complex_id'].values:
    cid2wt[c] = False